# gRPC Client Test

This notebook tests the gRPC rendering server by:
1. Loading camera data from USDZ file
2. Getting the first camera pose
3. Calling the gRPC API to render
4. Displaying the result with matplotlib

In [ ]:
from pathlib import Path

# Configuration
USDZ_PATH = Path(
    "data/sample_set/25.07_release/Batch0001/0d79a6a0-4aa7-4eb1-aea1-31fd55bd71d7/0d79a6a0-4aa7-4eb1-aea1-31fd55bd71d7.usdz"
)  # TODO: Update this path
SERVER_HOST = "127.0.0.1"
SERVER_PORT = 50051

In [ ]:
# Import gRPC dependencies
# Import gRPC protocol definitions
import sys

import matplotlib.pyplot as plt
import numpy as np

import grpc

sys.path.insert(0, str(Path.cwd().parent / "grpc"))

import simple_nurec_grpc
from simple_nurec_grpc import render_pb2, render_pb2_grpc

print("gRPC protocol loaded successfully")
print(f"Available modules: {simple_nurec_grpc.__all__}")

## Load Camera Data from USDZ

In [ ]:
from simple_nurec_viewer.export.camera import compute_camera_pose_NRE, get_camera_intrinsics, interpolate_rig_pose
from simple_nurec_viewer.export.loader import load_camera_data

# Load camera data
camera_calibrations, rig_trajectories, world_to_nre = load_camera_data(USDZ_PATH)

print(f"Loaded {len(camera_calibrations)} camera calibrations")
print(f"Rig trajectory sequence: {rig_trajectories.sequence_id}")
print(f"Number of rig poses: {len(rig_trajectories.T_rig_worlds)}")

# List all cameras
camera_names = list(set(calib.logical_sensor_name for calib in camera_calibrations.values()))
print(f"\nAvailable cameras: {camera_names}")

## Get First Camera Pose

In [ ]:
# Get the first camera (prefer front_wide if available)
camera_name = "camera_front_wide_120fov"

# Find the calibration key for this camera
calib_key = None
for key, calib in camera_calibrations.items():
    if calib.logical_sensor_name == camera_name:
        calib_key = key
        break

if calib_key is None:
    raise ValueError(f"Camera {camera_name} not found in calibrations")

calib = camera_calibrations[calib_key]

# Get first frame timestamp
if calib_key not in rig_trajectories.cameras_frame_timestamps_us:
    raise ValueError(f"No timestamps found for camera {camera_name}")

timestamps = rig_trajectories.cameras_frame_timestamps_us[calib_key]
first_timestamp_us = timestamps[0][0]  # (start_us, end_us)

print(f"Selected camera: {camera_name}")
print(f"Calibration key: {calib_key}")
print(f"Camera model: {calib.camera_model_type}")
print(f"Resolution: {calib.resolution}")
print(f"Principal point: {calib.principal_point}")
print(f"First timestamp: {first_timestamp_us} us")

In [ ]:
# Interpolate rig pose to camera timestamp
T_rig_world = interpolate_rig_pose(
    first_timestamp_us,
    rig_trajectories.T_rig_world_timestamps_us,
    rig_trajectories.T_rig_worlds,
)

# Compute camera pose in NRE coordinates
T_camera_NRE = compute_camera_pose_NRE(
    calib.T_sensor_rig,
    T_rig_world,
    world_to_nre,
)

# Get camera intrinsics
if calib.camera_model_type == "ftheta":
    K = get_camera_intrinsics(
        calib.camera_model_type,
        calib.resolution,
        calib.principal_point,
        max_angle=calib.max_angle,
    )
elif calib.camera_model_type == "pinhole":
    focal_length = getattr(calib, "focal_length", calib.resolution[0] * 0.8)
    K = get_camera_intrinsics(
        calib.camera_model_type,
        calib.resolution,
        calib.principal_point,
        focal_length=focal_length,
    )

print(f"Camera position in NRE: {T_camera_NRE[:3, 3]}")
print(f"\nCamera intrinsics K:\n{K}")

## Build gRPC Render Request

In [ ]:
# Build view matrix for gRPC rendering
# T_camera_NRE transforms from camera to NRE
# The server expects us to send T_camera_NRE directly, then it will compute viewmat = inv(T_camera_NRE)
camera_to_world_3x4 = T_camera_NRE[:3, :4].flatten().tolist()

# Extract intrinsics
fx, fy = K[0, 0], K[1, 1]
cx, cy = K[0, 2], K[1, 2]
width, height = calib.resolution

# Convert timestamp to seconds
timestamp_s = first_timestamp_us / 1e6

# Create render request
request = render_pb2.RenderRequest()
request.camera.camera_to_world.extend(camera_to_world_3x4)
request.camera.fx = fx
request.camera.fy = fy
request.camera.cx = cx
request.camera.cy = cy
request.camera.width = width
request.camera.height = height
request.camera.camera_model = calib.camera_model_type
request.camera.time = timestamp_s

# Add FTheta distortion parameters if applicable
if calib.camera_model_type == "ftheta":
    ftheta_params = request.camera.ftheta_params
    ftheta_params.reference_poly = calib.reference_poly
    ftheta_params.pixeldist_to_angle_poly.extend(calib.pixeldist_to_angle_poly)
    ftheta_params.angle_to_pixeldist_poly.extend(calib.angle_to_pixeldist_poly)
    ftheta_params.max_angle = calib.max_angle
    # Set linear_cde (C, D, E distortion coefficients)
    if hasattr(calib, "linear_cde") and calib.linear_cde is not None:
        ftheta_params.linear_cde.extend(calib.linear_cde)

print("\nRender request created:")
print(f"  Camera: {camera_name}")
print(f"  Model: {calib.camera_model_type}")
print(f"  Resolution: {width}x{height}")
print(f"  Intrinsics: fx={fx:.1f}, fy={fy:.1f}, cx={cx:.1f}, cy={cy:.1f}")
print(f"  Timestamp: {timestamp_s:.3f}s")

## Connect to gRPC Server and Render

In [ ]:
# Create gRPC channel with increased message size (256 MB for high-res images)
max_message_length = 256 * 1024 * 1024  # 256 MB
channel_options = [
    ("grpc.max_send_message_length", max_message_length),
    ("grpc.max_receive_message_length", max_message_length),
]
channel = grpc.insecure_channel(f"{SERVER_HOST}:{SERVER_PORT}", options=channel_options)

# Create stub
stub = render_pb2_grpc.RenderServiceStub(channel)

print(f"Connected to gRPC server at {SERVER_HOST}:{SERVER_PORT}")

In [ ]:
# Send render request
print(f"Sending render request for {camera_name}...")

try:
    response = stub.Render(request)

    if response.success:
        print("Render successful!")
        print(f"  Resolution: {response.rgb_image.width}x{response.rgb_image.height}")
        print(f"  Render time: {response.render_time_ms:.2f}ms")
        print(f"  Data size: {len(response.rgb_image.rgb_data)} bytes")
    else:
        print(f"Render failed: {response.error_message}")
except grpc.RpcError as e:
    print(f"gRPC error: {e.code()}")
    print(f"Details: {e.details()}")

## Display Rendered Image

In [ ]:
if response.success:
    # Convert bytes to numpy array
    rgb_data = np.frombuffer(response.rgb_image.rgb_data, dtype=np.uint8)
    image = rgb_data.reshape(response.rgb_image.height, response.rgb_image.width, 3)

    # Display image
    plt.figure(figsize=(12, 8))
    plt.imshow(image)
    plt.title(f"{camera_name} - {calib.camera_model_type} - t={timestamp_s:.3f}s")
    plt.axis("off")
    plt.tight_layout()
    plt.show()

    # Print image info
    print(f"\nImage shape: {image.shape}")
    print(f"Data type: {image.dtype}")
    print(f"Value range: [{image.min()}, {image.max()}]")
else:
    print("No image to display (render failed)")

## Cleanup

In [ ]:
# Close channel
channel.close()
print("Channel closed")